In [1]:
from pyomo_orm.core.database import Session, bind_engine
from pyomo_orm.core.utils import as_dataframe

from models import *

In [2]:
bind_engine('sqlite:///diet.sqlite')

Engine(sqlite:///diet.sqlite)

In [3]:
from pyomo_orm.core.problems import BaseProblem
from pyomo_orm.core.wrappers import *
infinity = float('inf')

In [4]:
from pyomo.environ import NonNegativeReals

class DietProblem(BaseProblem):
    # Sets
    foods = ORMSet(
        model=Food,
        queryset=Food.query().filter(Food.problem_run_id is None)
    )
    nutrients = ORMSet(
        model=Nutrient,
        queryset=Nutrient.query().filter(Nutrient.problem_run_id is None)
    )

    # Params
    cost = ORMParam(
        'foods',
        model=Food,
        from_attr='cost'
    )
    volume_per_serving = ORMParam(
        'foods',
        model=Food,
        from_attr='volume_per_serving'
    )
    amount = ORMParam(
        'foods',
        'nutrients',
        model=FoodNutrientAmount,
        from_attr='amount',
        indexed_by=['food_id', 'nutrient_id']
    )
    nutrient_lower_bound = ORMParam(
        'nutrients',
        model=Nutrient,
        from_attr='lower_bound',
        within=NonNegativeReals,
        default=0.0
    )
    nutrient_upper_bound = ORMParam(
        'nutrients',
        model=Nutrient,
        from_attr='upper_bound',
        within=NonNegativeReals,
        default=infinity
    )


In [5]:
diet = DietProblem(
    name='Diet Problem', 
    description="""
From a selection of foods curate a meal plan that meets nutrient requirements at lowest cost
"""
)

In [6]:
diet.define_sets()

In [7]:
diet.define_params()

In [8]:
diet.pyomo_model.__dict__

{'_constructed': False,
 '_ctypes': {pyomo.core.base.sets.Set: [0, 1, 2],
  pyomo.core.base.param.Param: [2, 6, 5]},
 '_data': {None: <pyomo.core.base.PyomoModel.AbstractModel at 0x2a658237510>},
 '_decl': {'amount': 2,
  'cost': 3,
  'foods': 0,
  'nutrient_lower_bound': 4,
  'nutrient_upper_bound': 5,
  'nutrients': 1,
  'volume_per_serving': 6},
 '_decl_order': [(<pyomo.core.base.sets.SimpleSet at 0x2a658286818>, 1),
  (<pyomo.core.base.sets.SimpleSet at 0x2a6582869a8>, None),
  (<pyomo.core.base.param.SimpleParam at 0x2a658232870>, 3),
  (<pyomo.core.base.param.SimpleParam at 0x2a655916630>, 4),
  (<pyomo.core.base.param.SimpleParam at 0x2a658232990>, 5),
  (<pyomo.core.base.param.SimpleParam at 0x2a658232240>, 6),
  (<pyomo.core.base.param.SimpleParam at 0x2a658232ab0>, None)],
 '_implicit_subsets': None,
 '_index': {None},
 '_name': 'unknown',
 '_options': None,
 '_parent': None,
 '_rule': None,
 '_suppress_ctypes': set(),
 '_type': pyomo.core.base.block.Block,
 'amount': <pyomo.